In [1]:
import numpy as np

In [2]:
import scipy
from scipy.io import loadmat
from data_generator import DataGenerator

In [4]:
from likelihood import GaussianLogLikelihood
from prior import ComplexityLogPrior
from posterior import LogPosterior
from mcmc import MCMC
from sequential_mc import SequentialMC
from util import generate_binary_coef

In [5]:
import matplotlib.pyplot as plt

In [6]:
data = loadmat('../data/phase_field_oneD_simulation_beginning_stage.mat')

In [5]:
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 't', 'x', 'usol'])

In [6]:
u = data['usol']
t = np.squeeze(data['t'])
x = np.squeeze(data['x'])

In [7]:
from numerical_derivative import ChebyshevLocalFit_1D

In [8]:
width = 5

In [9]:
du_x, du_xx, du_t = ChebyshevLocalFit_1D(u, x, t, deg=3, width=width, diff=2)

In [10]:
u = u[width:(-width), width:(-width)]

In [11]:
n_samples = 5000

In [12]:
u = u.flatten()
du_x = du_x.flatten()
du_xx = du_xx.flatten()
du_t = du_t.flatten()

In [13]:
data = np.vstack([u,du_x,du_xx, du_t]).T

In [14]:
data.shape

(240100, 4)

In [15]:
np.random.seed(1001)

In [16]:
np.random.shuffle(data)

In [30]:
a = '22'

In [32]:
isinstance(a,str)

True

In [17]:
data = data[:n_samples]

In [18]:
data.shape

(5000, 4)

In [19]:
np.save('raw_data', data)

In [7]:
data = np.load('raw_data.npy')

In [8]:
dg = DataGenerator()

In [9]:
data[:,:-1].shape

(5000, 3)

In [10]:
X, names = dg(data[:,:-1], descriptions=['u','u_x','u_xx'] )

In [11]:
Y = data[:,-1]

In [20]:
prior = ComplexityLogPrior(method='num_terms', simplicity_preference=1.)

In [21]:
likelihood = GaussianLogLikelihood(X,Y)

In [22]:
posterior = LogPosterior(prior, likelihood)

In [23]:
import itertools

In [24]:
init_coefs = generate_binary_coef(1, len(names))

In [25]:
sequentialmc = SequentialMC(posterior=posterior,log_file='mc.csv')

In [26]:
names

[u,
 u_x,
 u_xx,
 u**2,
 u*u_x,
 u*u_xx,
 u_x**2,
 u_x*u_xx,
 u_xx**2,
 u**3,
 u**2*u_x,
 u**2*u_xx,
 u*u_x**2,
 u*u_x*u_xx,
 u*u_xx**2,
 u_x**3,
 u_x**2*u_xx,
 u_x*u_xx**2,
 u_xx**3]

In [38]:
len(names)

19

In [39]:
init_coefs[0] = np.array([1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0])

In [ ]:
samples = sequentialmc(samples=init_coefs,beta0_nsteps=10,beta0to1_nsteps=100,beta1_nsteps=200, descriptions=names )

step: 0, beta:  0.000
step: 1, beta:  0.000
step: 2, beta:  0.000
step: 3, beta:  0.000
step: 4, beta:  0.000
step: 5, beta:  0.000
step: 6, beta:  0.000
step: 7, beta:  0.000
step: 8, beta:  0.000
step: 9, beta:  0.000
step: 10, beta:  0.000
step: 11, beta:  0.010
step: 12, beta:  0.020
step: 13, beta:  0.030
step: 14, beta:  0.040
step: 15, beta:  0.051
step: 16, beta:  0.061
step: 17, beta:  0.071
step: 18, beta:  0.081
step: 19, beta:  0.091
step: 20, beta:  0.101
step: 21, beta:  0.111
step: 22, beta:  0.121
step: 23, beta:  0.131
step: 24, beta:  0.141
step: 25, beta:  0.152
step: 26, beta:  0.162
step: 27, beta:  0.172
step: 28, beta:  0.182
step: 29, beta:  0.192
step: 30, beta:  0.202
step: 31, beta:  0.212
step: 32, beta:  0.222
step: 33, beta:  0.232
step: 34, beta:  0.242
step: 35, beta:  0.253
step: 36, beta:  0.263
step: 37, beta:  0.273
step: 38, beta:  0.283
step: 39, beta:  0.293
step: 40, beta:  0.303
step: 41, beta:  0.313
step: 42, beta:  0.323
step: 43, beta:  0.33

In [29]:
import pandas as pd

In [32]:
data = pd.read_csv('mc.csv', header=0)

In [33]:
data

,step,sample 0,sample 1,sample 2,sample 3,sample 4,sample 5,sample 6,sample 7,sample 8,...,sample 10,sample 11,sample 12,sample 13,sample 14,sample 15,sample 16,sample 17,sample 18,sample 19
0,0,0,0.0256136998623041*u - 0.0189419221597271,0.0083895697790101 - 3.9822356165047*u_x,8.5709133879994e-5 - 19.8486850396247*u_xx,0.0310305444817164*u**2 - 0.0221055406291764,-5.41989886281898*u*u_x + 0.00565334578072126,1.65304638915195*u*u_xx + 5.65292054203851e-5,0.00425856832010691 - 124.509510070196*u_x**2,-657.712714747617*u_x*u_xx + 5.72276961979775e-5,...,0.0327621273276123*u**3 - 0.0228643269344457,-5.50093184529217*u**2*u_x + 0.0038319466130507,19.1945559633285*u**2*u_xx + 0.000640500815464303,-211.887289315214*u*u_x**2 + 0.00341437565824889,342.377303674869*u*u_x*u_xx + 0.00013552948880...,-25411.7525562607*u*u_xx**2 + 0.00135517150449133,0.00249037364403976 - 3175.02825596852*u_x**3,-18314.4709593414*u_x**2*u_xx + 3.258908924745...,-752565.661938054*u_x*u_xx**2 + 0.001828005828...,8.72222312416797e-5 - 2857906.26319175*u_xx**3
1,1,0.796365736741299*u**3 - 3.15891432628123*u**2...,24.6093628003228*u**2*u_x + 144.333050271164*u...,-1.01212217455373*u**3 - 0.264787542798352*u**...,0.498709817083913*u**3 + 34.5800283701242*u**2...,-82.3162037325259*u**2*u_xx - 53.5171995127869...,35.413980550162*u**2*u_x + 1380.02087244515*u*...,1.05052310752068*u**3 + 81.4562983877699*u**2*...,-1.00695001108192*u**3 + 1.90877540075048*u**2...,0.473859515272678*u**3 - 0.463152609386418*u**...,...,-1.00401428023207*u**3 - 0.04921140981297*u**2...,1.13932454392277*u**3 + 9.99802309943714*u**2*...,0.307621120821782*u**3 - 944.893417974198*u*u_...,1010.59981251244*u**2*u_xx + 415.414247987206*...,814.872007505346*u*u_x*u_xx - 12.3663896279605...,39.2969758730904*u**2*u_x - 155.642497011924*u...,0.782732592223496*u**3 + 28.7363379961748*u**2...,0.626042249430342*u**3 + 1047.91731480456*u**2...,0.905836395570688*u**2 - 530.161894721049*u*u_...,-1.00687719874346*u**3 - 0.569239876455052*u**...
2,2,7.60768947191288*u**2*u_x - 17829.8130616442*u...,0.0151112149883552*u**2 + 69.874609616469*u*u_...,0.0178974504716337*u**2 + 2187.12414179433*u*u...,1.84177518517871*u**3 + 57.5317120184227*u**2*...,20.3517144686202*u**2*u_x - 493.347386071755*u...,0.167524220511345*u**3 - 253201.200391612*u*u_...,0.289133426977548*u**2 + 64636.7427087526*u*u_...,0.511077612051755*u**2 - 75.2124440862532*u*u_...,0.00484950345808942*u**3 + 99.6824262970693*u*...,...,72.9592185702302*u**2*u_x + 1507.81232448167*u...,0.330413351844699*u**3 + 4.78909755053358*u**2...,-0.965824610777657*u**3 + 17.1466474826346*u**...,-41.8675689249819*u**2*u_xx + 7345.24015764468...,-15.5376871356297*u**2*u_x - 247.363631071785*...,32.8688919697722*u**2*u_x - 5630.53681139102*u...,0.482575808319259*u**3 + 18.4893389400513*u**2...,83.5455275183589*u**2*u_x + 567.682458183691*u...,-1.006179850792*u**3 - 0.319067166296712*u**2*...,1454.59572240575*u**2*u_xx + 55796.5784290934*...
3,3,0.0123653417956881*u**2 + 2668.40280336591*u*u...,-1.02052070570187*u**3 - 0.51958605124456*u**2...,-0.959276700476131*u**3 + 0.275633404004468*u*...,0.288186881085897*u**3 - 139.634478420815*u**2...,0.469733781506759*u**3 - 1.30573316184503*u**2...,0.574492350254202*u**3 - 3.27126951548962*u**2...,-310.841953466875*u**2*u_xx - 9170.84919197733...,0.0134049847109928*u**2 + 1950.2366885655*u*u_...,0.343958877644583*u**3 + 5.26132093614344*u**2...,...,39.6124623364175*u**2*u_x + 154.230040504564*u...,-1.00267586850883*u**3 + 1.90424799818692*u**2...,730.403643192005*u**2*u_xx - 247.789942779155*...,0.470055047957533*u**3 + 2298.35896623321*u*u_...,0.0133552552716404*u**3 - 21.030625765639*u**2...,0.276116731211512*u**3 + 883.91229658212*u**2*...,0.262637910992894*u**3 + 450.878263236236*u**2...,-1.10986522821008*u**3 - 2.79617535552356*u**2...,-1.0031596404671*u**3 + 0.443595573595466*u**2...,-1.5666451963817*u**2*u_x - 54.1144596456011*u...
4,4,-16.129105439918*u**2*u_x - 121212.014010917*u...,-1.03252922150293*u**3 + 1.0

In [37]:
data.values[-1]

array([400,
       '455.018268555224*u**2*u_xx + 0.629012726134742*u**2 + 8.30864405201357*u*u_x - 478.540689427039*u*u_xx - 0.626166816453172*u - 2013.18239741741*u_x**3 - 97284.0893497997*u_x**2*u_xx + 556281.759580422*u_x*u_xx**2 + 7340.42502564581*u_x*u_xx - 25870.0647088849*u_xx**2 - 12.084520400237*u_xx + 0.00813896773105892',
       '0.773738710821334*u**3 - 0.757819925117817*u**2 - 2735.03320803586*u*u_x*u_xx + 9.33531268250373*u*u_x - 83364.8025086424*u*u_xx**2 + 175119.326273052*u_x**2*u_xx - 163.314757582846*u_x**2 + 1231153.18025563*u_x*u_xx**2 - 4995.58785983648*u_x*u_xx - 0.00378389167553842',
       '1.84403143791399*u**3 + 51.8152318584333*u**2*u_x - 1.83894383782383*u**2 - 526.052218633697*u*u_x**2 - 234071.969992467*u*u_xx**2 + 328.739487426873*u*u_xx + 135674.020322036*u_x**2*u_xx - 3540.38088552918*u_x*u_xx - 20535811.1332133*u_xx**3 + 124050.323791814*u_xx**2 - 138.251077664269*u_xx + 0.00513366227607125',
       '945.719275253651*u**2*u_xx - 3205.76134276949*u*u_x

In [17]:
data = pd.read_csv("mc.log", header=0, delim_whitespace=True)

In [18]:
pd.set_option('display.max_columns', None)

In [19]:
data

,step,weight_1,weight_2,weight_3,weight_4,weight_5,weight_6,weight_7,weight_8,weight_9,weight_10,weight_11,weight_12,weight_13,weight_14,weight_15,weight_16,weight_17,weight_18,weight_19,bias,accp_prob,logPrior,logLikelihood,logPosterior
0,0,-0.902409,0.0,10.296157,1.907862,0.0,0.0,0.0,0.0,0.0,-1.00538,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.009829,1.0,-0.0,-0.215,-0.215


In [20]:
type(names[0])

sympy.core.symbol.Symbol

In [31]:
a= np.array([[1,2,3,1],[1,2,3,1]])

In [32]:
a

array([[1, 2, 3, 1],
       [1, 2, 3, 1]])

In [33]:
np.unique(a,return_counts=True,axis=1)

(array([[1, 2, 3],
        [1, 2, 3]]), array([2, 1, 1]))

In [34]:
np.zeros(100)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [37]:
np.linspace(0,1,10)

array([0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444,
       0.55555556, 0.66666667, 0.77777778, 0.88888889, 1.        ])